
#Lab3 : Morphology

Authors:<br>
* Ramón Mateo Navarro
* Benet Manzanares Salor

##Installation ans imports

In [ ]:
import os
import pandas as pd
import nltk

from argparse import Namespace
from google.colab import drive
from scipy.stats import pearsonr
from nltk.metrics import jaccard_distance
from nltk.stem import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Settings


In [ ]:
settings = Namespace()

settings.mount_path = "/content/drive/"
drive.mount(settings.mount_path, force_remount=True)

settings.project_folder = "IHTL_Labs/Lab3" #@param {type:"string"}
settings.project_path = os.path.join(os.path.join(settings.mount_path, "MyDrive"), settings.project_folder)
settings.input_filename = "STS.input.SMTeuroparl.txt" #@param {type:"string"}
settings.input_filepath = os.path.join(settings.project_path, settings.input_filename)
settings.gs_filename = "STS.gs.SMTeuroparl.txt" #@param {type:"string"}
settings.gs_filepath = os.path.join(settings.project_path, settings.gs_filename)

Mounted at /content/drive/


In [ ]:
wnl = WordNetLemmatizer()


## Data

In [ ]:
dt = pd.read_csv(settings.input_filepath,sep='\t', header=None)
dt['gs'] = pd.read_csv(settings.gs_filepath, sep='\t', header=None)
dt.head()


,0,1,gs
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.50
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.25
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.50
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.00


## Experiment

* Tokenizing words

In [ ]:
s0_list = [nltk.word_tokenize(dt[0][s]) for s in range(len(dt[0]))]
s1_list = [nltk.word_tokenize(dt[1][s]) for s in range(len(dt[1]))]

## all words tokenized in this moment

* Lemmas

In [ ]:
s0_list_tags = [nltk.pos_tag(words) for words in s0_list] 
s1_list_tags = [nltk.pos_tag(words) for words in s1_list]

def lemmatize(p):
    if p[1][0] in {'N','V'}:
        return wnl.lemmatize(p[0].lower(), pos=p[1][0].lower())
    return p[0]

r0 = []
r1 = []

for i in range(len(s0_list_tags)):
  r0.append([lemmatize(pair) for pair in s0_list_tags[i]])
  r1.append([lemmatize(pair) for pair in s1_list_tags[i]])


* Jaccard distance 

In [ ]:
def jaccard_similariy(sentences):
  sent1, sent2 = sentences
  words1 = set(nltk.word_tokenize(sent1))
  words2 = set(nltk.word_tokenize(sent2))
  return 1-jaccard_distance(words1, words2)

dt["jaccard"] = [1 -jaccard_distance(set(r0[x]), set(r1[x])) for x in range(len(s0_list))]


In [ ]:
pearsonr(dt['gs'], dt['jaccard'])[0]

0.4569107458417673

In [ ]:
dt[:25]

,0,1,gs,jaccard
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.500,0.346154
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.000,0.923077
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.250,0.391304
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.500,0.545455
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.000,1.000000
5,Neither was there a qualified majority within ...,There was not a majority voting in Parliament ...,5.000,0.333333
6,It increases the power of the big countries at...,It has the effect of augmenting the potency of...,4.667,0.333333
7,"The fishermen are inactive, tired and disappoi...","The fishers are inactive, tired and disappointed.",5.000,0.800000
8,The leaders have now been given a new chance a...,Leaders now have another chance to let them an...,5.000,0.318182
9,The vote will take place today at 5.30 p.m.,The vote will take place at 17h30.,5.000,0.636364
